## Yield calculator.

This notebook contains the code to calculate yields from e.g. Battino et a. (2019, 2021) or Ritter et al. (2018) MESA models + stellar postprocessing.

The yields we use here are the integrated total ejected mass yield, i.e.

$$
EM = \int X_{i, \rm surf}(t)\,\dot{M}(t)\ dt
$$

To calculate this integral, we simply sum over the surface abundance (times the change in mass) for each timestep of the star. We additionally add the mass contained outside the final reminant at the final snapshot time as this should be included as well.

In [ ]:
# default data location (try mirror if default
# is not available)
import matplotlib.pyplot as plt
from nugridpy import mesa as ms
from nugridpy import nugridse as mp

In [ ]:
import contextlib
import io
import sys

In [ ]:
import numpy as np

### The MESA stellar evolution model

By default MESA is putting out two types of data. History data provides the time evolution of scalar quantities, one per time step. This data can be accessed with the `mesa.star_log` (or `mesa.history_data` which is the same) class.

MESA also outputs profile data at select time steps. Profiles are available via the `mesa_profile` class.

#### History data
Initialise the 2 solar-mass Z=0.02 MESA stellar evolution model from set1.2 using the seeker method:

In [ ]:
setname = "set1.01"

In [ ]:
modelname = "m2z1m3"

In [ ]:
def calc_wind_yields(mass, iso_massf):
    EM = np.zeros(len(iso_massf[0]))
    dm = -np.diff(mass)

    for i in range(len(mass) - 1):
        print(f"{i} / {len(mass)}", end="\r")
        X = iso_massf[i + 1]
        EM += dm[i] * X

    return EM

In [ ]:
def get_path(modelname):
    if "z2m2" in modelname:
        setname = "set1.2"
    elif "z3m2" in modelname:
        setname = "set1.3"
    elif "z1m2" in modelname:
        setname = "set1.1"
    elif "z2m3" in modelname:
        setname = "set1.02"
    elif "z1m3" in modelname:
        setname = "set1.01"
    else:
        raise Exception("set not found")

    if "set1.0" in setname:
        path = f"/data/nugrid_data/set1upd/{setname}/ppd_wind/{modelname}/"
    else:
        path = f"/data/nugrid_data/set1upd/{setname}/ppd_wind/RUN_set1upd_{modelname}/"

    return path

In [ ]:
def calc_all(modelname, *, H_min=0.01, tol=1e-3):
    path = get_path(modelname)
    print("reading ", path + "/H5_out")
    pt = mp.se(sedir=f"{path}/H5_out/")
    pt_surf = mp.se(sedir=f"{path}/H5_surf/")
    print("read all files")

    isotopes = pt.se.isotopes
    assert isotopes[1] == "H-1"  # need for later...

    print("reading arrays")
    Xs_surf = pt_surf.get("iso_massf")
    mass = pt_surf.get("mass")

    print("calculating wind yields")
    EM_wind = calc_wind_yields(mass, Xs_surf)
    m_ej = mass[0] - mass[-1]

    if abs(1 - m_ej / np.sum(EM_wind)) > tol:
        print("mass mismatch")
        print("yield sum: ", EM_wind)
        print("expected", m_ej)

    print("calculating reminant yields")
    cyclemax = np.int64(pt.se.cycles[-1])
    m_end = pt.get(cyclemax, "mass")
    Xs_end = pt.get(cyclemax, "iso_massf")

    m_rem = m_end[Xs_end[:, 1] > H_min][0]

    rem_yield = calc_rem_yield(pt, m_rem)

    tot_yield = rem_yield + EM_wind

    if abs(1 - (np.sum(tot_yield) + m_rem) / mass[0]) > tol:
        print("mass loss from yields ", np.sum(tot_yield))
        print("mini - mrem", mass[0] - m_rem)

    filename = f"yields_{modelname}.txt"
    Xi = Xs_surf[0]

    if np.abs(np.sum(Xi) - 1) > tol:
        print("surface initial abundance sum:", np.sum(Xi))
    print("writing to file ", filename)

    write_header(filename, modelname, setname, mass, m_rem)
    write_yields(filename, isotopes, tot_yield, Xi)

    return tot_yield

In [ ]:
def calc_rem_yield(pt, m_rem):
    cyclemax = np.int64(pt.se.cycles[-1])
    mass = pt.get(cyclemax, "mass")
    filt = mass > m_rem
    
    f = io.StringIO()
    Xs = pt.get(cyclemax, "iso_massf")[filt, :]
    
    dm = np.gradient(mass)[filt]
    
    ele_mass = Xs * np.reshape(dm, (-1, 1))

    
    return np.sum(ele_mass, axis=0)

In [ ]:
from datetime import datetime

In [ ]:
def write_header(filename, modelname, setname, mass, m_rem):
    with open(filename, "w") as file:
        print("# Battino yields", file=file)
        print(f"# Model: {modelname}", file=file)
        print(f"# Set: {setname}", file=file)
        print(f"# Mini: {mass[0]:0.2f}", file=file)
        print(f"# Mfinal: {m_rem}", file=file)
        print(
            f"# Created by Daniel Boyea on {datetime.today().strftime('%Y-%m-%d')}",
            file=file,
        )
        print(f"{'isotope':10}    {'mass_yield':8}     {'X0':8}", file=file)

In [ ]:
def write_yields(filename, isotopes, yields, Xini):
    with open(filename, "a") as file:
        for i in range(len(yields)):
            print(
                f"{isotopes[i]:10}     {yields[i]:8.4e}     {Xini[i]:8.4e}", file=file
            )

In [ ]:
calc_all("m2z2m2")

In [ ]:
calc_all("m3z2m2")

In [ ]:
calc_all("m2z1m2")

In [ ]:
calc_all("m3z1m2")

## low z models

In [ ]:
def calc_all_lowz(modelname, *, H_min=0.01, tol=1e-3):
    pt_surf = mp.se(
        sedir=f"{modelname}/H5_surf"
    )
    sl = ms.history_data(
        f"{modelname}/LOGS",
    )
    print("read all files")

    isotopes = pt_surf.se.isotopes
    assert isotopes[1] == "H-1"  # need for later...

    print("reading arrays")
    Xs_surf = pt_surf.get("iso_massf")
    mass = pt_surf.get("mass")

    print("calculating wind yields")
    EM_wind = calc_wind_yields(mass, Xs_surf)
    m_rem = sl.get("h1_boundary_mass")[-1]
    m_ej_normal = mass[0] - mass[-1]

    if abs(1 - m_ej_normal / np.sum(EM_wind)) > tol:
        print("mass mismatch")
        print("yield sum: ", EM_wind)
        print("expected", m_ej_normal)

    print("calculating reminant yields")
    Xi = Xs_surf[0]
    Xf = Xs_surf[-1]

    rem_yield = (mass[-1] - m_rem) * Xf
    
    tot_yield = rem_yield + EM_wind

    if abs(1 - (np.sum(tot_yield) + m_rem) / mass[0]) > tol:
        print("mass loss from yields ", np.sum(tot_yield))
        print("mini - mrem", mass[0] - m_rem)

    filename = f"yields_{modelname}.txt"

    if np.abs(np.sum(Xi) - 1) > tol:
        print("surface initial abundance sum:", np.sum(Xi))
    print("writing to file ", filename)

    write_header(filename, modelname, setname, mass, m_rem)
    write_yields(filename, isotopes, tot_yield, Xi)

    return tot_yield

In [ ]:
calc_all("m3z1m3-bigpoc")

In [ ]:
calc_all_lowz("m2z1m3")

In [ ]:
calc_all_lowz("m3z1m3")

In [ ]:
calc_all_lowz("m2z2m3")

## Surface abundance plots

In [ ]:
sys.path.append("../arya")
import arya

In [ ]:
setname = "set1.2"

In [ ]:
modelname = "m2z2m2"

In [ ]:
pt = mp.se(
    sedir=f"/data/nugrid_data/set1upd/{setname}/ppd_wind/RUN_set1upd_{modelname}/H5_out/"
)

In [ ]:
pt_surf = mp.se(
    sedir=f"/data/nugrid_data/set1upd/{setname}/ppd_wind/RUN_set1upd_{modelname}/H5_surf/"
)

In [ ]:
model_number = [int(c) for c in pt_surf.se.cycles[::10]]

In [ ]:
Xs_surf = np.array(pt_surf.get(model_number, "iso_massf"))

In [ ]:
m_star = pt_surf.get(model_number, "mass")

In [ ]:
isotopes = np.array(pt_surf.se.isotopes)

In [ ]:
model_number = np.int64(model_number)

In [ ]:
model_number

In [ ]:
for iso in isotopes:
    print(iso)

In [ ]:
def plot_isos(isos, relative=True):
    for (i, ele) in enumerate(isos):
        ii = np.where(isotopes == ele)[0]
        if len(ii) == 1:
            i = ii[0]
            X = Xs_surf[:, i]
            if relative:
                log_X0 = np.log10(X[0])
            else:
                log_X0 = 0

            y = np.log10(X) - log_X0
            plt.plot(model_number, y, label=ele, ls=["-", "--", ":", "-."][i % 4])
        else:
            print("element not found", ele)

    plt.legend(bbox_to_anchor=(1, 1), loc="upper left")
    plt.xlabel("model number")
    plt.ylabel("log X / Xi")
    plt.show()

In [ ]:
isos = ["C-12", "C-13", "O-16", "Pb-206"]
plot_isos(isos)

In [ ]:
isos = ["Al-27", "P-31", "K-39"]
plot_isos(isos)

In [ ]:
isos = ["Mg-24"]
plot_isos(isos)

In [ ]:
isos = ["S-32", "Cl-35", "Ar-36", "Ca-40", "V-51", "Fe-56"]
plot_isos(isos)

In [ ]:
pt.abu_profile(isos=isos, fname=model_number[-1], logy=True, colourblind=True)
pt.abu_profile(isos=isos, fname=model_number[0], logy=True, colourblind=True)

In [ ]:
10 ** -0.003

In [ ]:
isos = ["Co-59", "Zn-64"]
plot_isos(isos)

In [ ]:
len(Xs[0])

In [ ]:
Xs[0] == Xs[300]